# Work with Videos

**[Video](https://docs.aperturedata.io/category/video-commands) is an object recognized by ApertureDB and the query language allows you to directly add, find, update, and delete video.**

## Connect to the database

If you haven't already setup the database or configured it, check out our [quick start guide](https://docs.aperturedata.io/Setup/QuickStart)

In [ ]:
# Install the required client packages if needed
%pip install --upgrade --quiet pip
%pip install --upgrade --quiet aperturedb

In [1]:
from aperturedb.CommonLibrary import create_connector

# Create the connector for ApertureDB
client = create_connector()

In [2]:
# Simple query to see how the database is doing
# https://docs.aperturedata.io/query_language/Reference/db_commands/GetStatus
query = [{
    "GetStatus": {
    }
}]

# Execute the query to get back a JSON response for GetStatus 
response, blobs = client.query(query)

client.print_last_response()

[
    {
        "GetStatus": {
            "info": "OK",
            "status": 0,
            "system": "ApertureDB",
            "version": "0.17.23"
        }
    }
]


## Create or Add a Cooking Video to ApertureDB

Let's say we want to add some videos of how some recipe is prepared. One way to introduce new videos in the database is through our query language

**For bulk additions, we recommend using the [Python SDK loaders](http://docs.aperturedata.io/category/manage-multimodal-datasets)**

In [ ]:
# Download the sample file
! mkdir -p data; cd data; wget https://github.com/aperture-data/Cookbook/blob/e333f6c59070b9165033d9ddd5af852a6b9624ba/notebooks/simple/data/crepe_flambe.mp4; cd -

In [3]:
query = [{
    "AddVideo": {
        # Notice the missing "class" property since we already know its a Video (represented as _Video in ApertureDB)
        "properties": {
            "name": "crepe_flambe",
            "id": 45,
            "category": "dessert",
            "cuisine": "French",
            "location": "Brittany",
            "caption": "Special Brittany flambe crepe"
        },
        "if_not_found": {                  # avoid adding twice
            "id": ["==", 45]
        }
    }
}]

# Read the image data as a binary blob
fd = open("data/crepe_flambe.mp4", 'rb')
array = [ fd.read() ]
fd.close()

response, blobs = client.query(query, array)

client.print_last_response()

[
    {
        "AddVideo": {
            "status": 0
        }
    }
]


## Query video by its metadata attributes

Verify this Video was added to the database and read all the property values

In [ ]:
from aperturedb import NotebookHelpers as nh   # Our helper package for image displays and other utilities

query = [{
    "FindVideo": {
        "constraints": {
            "name": [">=", "crepe"],
            "location": [">", "A"] 
        },
        "blobs": True,                          # This is set to False by default
        "results": {
            "all_properties": True
        }
    }
}]

response, blobs = client.query(query)

client.print_last_response()
num_videos = response[0]["FindVideo"]["returned"]
for count in range(num_videos):
    nh.display_video_mp4(blobs[count])

## Update properties of the video already in ApertureDB

Use [UpdateVideo](https://docs.aperturedata.io/query_language/Reference/video_commands/video_commands/UpdateVideo) if any of the attributes need a new value or your application now needs a new attribute in existing videos

In [5]:
query = [{
    "UpdateVideo": {
        "properties": {
            "contributor": "Vishakha"     # property will get added if missing or the value will be updated
        },
        "constraints": {
            "name": ["==", "crepe_flambe"]
        },
    }
}]

response, blobs = client.query(query)

client.print_last_response()

[
    {
        "UpdateVideo": {
            "count": 1,
            "status": 0
        }
    }
]


In [6]:
query = [{
    "FindVideo": {
        "constraints": {
            "name": ["==", "crepe_flambe"]
        },
        "results": {
            "all_properties": True
        }
    }
}]

response, blobs = client.query(query)

client.print_last_response()

[
    {
        "FindVideo": {
            "entities": [
                {
                    "_duration_us": 5500000,
                    "_fps": 30,
                    "_frame_count": 165,
                    "_frame_height": 480,
                    "_frame_width": 852,
                    "_uniqueid": "5.6863.224520",
                    "caption": "Special Brittany flambe crepe",
                    "category": "dessert",
                    "contributor": "Vishakha",
                    "cuisine": "French",
                    "id": 45,
                    "location": "Brittany",
                    "name": "crepe_flambe"
                }
            ],
            "returned": 1,
            "status": 0
        }
    }
]


## Delete the video we no longer need

In [7]:
query = [{
    "DeleteVideo": {
        "constraints": {
            "name": ["==", "crepe_flambe"]     # if this matches multiple videos, they will all be deleted
        }
    }
}]

response, blobs = client.query(query)

client.print_last_response()

[
    {
        "DeleteVideo": {
            "count": 1,
            "status": 0
        }
    }
]


### Verify deletion

We can verify that the video is not longer in the database.

In [8]:
query = [{
    "FindVideo": {
        "constraints": {
            "name": ["==", "crepe_flambe"]
        },
        "results": {
            "all_properties": True
        }
    }
}]

response, blobs = client.query(query)

client.print_last_response()

[
    {
        "FindVideo": {
            "returned": 0,
            "status": 0
        }
    }
]


## What's next?

* [Bulk load](https://docs.aperturedata.io/HowToGuides/Ingestion/Ingestion/Ingestion) videos
* Perform [operations on videos](https://docs.aperturedata.io/HowToGuides/start/Operations) or [add clips/frames](https://docs.aperturedata.io/HowToGuides/start/ClipsFrames) to ApertureDB